### 절대 모멘텀 
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM'별 마지막날의 데이터를 month_last_df 저장
- 전월의 기준이되는 컬럼의 값을 가진 파생변수 생성 
- 전년도의 기준이 되는 컬럼의 값을 가진 파생변수 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 거래 내역 생성
- 수익율 계산

In [ ]:
import pandas as pd
import numpy as np 
from datetime import datetime

In [ ]:
# 데이터 로드 
df = pd.read_csv("../../csv/AAPL.csv", index_col='Date')

In [ ]:
df.head(2)

In [ ]:
# index를 시계열 데이터로 변경 
df.index = pd.to_datetime(df.index, utc=True)

In [ ]:
# STD-YM 파생변수를 생성해서 index에서 년-월 추출 대입
# 컬럼의 데이터였다면 -> df['Date'].dt.strftime(format)
# index에서는 strftime() 바로 적용용
df['STD-YM'] = df.index.strftime('%Y-%m')

In [ ]:
df.iloc[10:20]

In [ ]:
# 월말의 데이터를 출력하는 방법 
# case1
df.loc['1981-1'].tail(1)
test_df = pd.DataFrame()
for year in range(1980, 2026): # 년도를 반복
    for month in range(1, 13): # 월을 반복
        try:
            idx = f"{year}-{month}"
            df2 = df.loc[idx].tail(1)
            test_df = pd.concat( [test_df, df2], axis=0 )
        except Exception as e:
            print(e)
            continue


In [ ]:
test_df

In [ ]:
# case2 
# STD-YM이용 : 현재의 STD-YM 데이터와 다음 인덱스의 STD-YM이 다른 경우
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag]

In [18]:
# case3 
# unique() 함수를 이용
ym_list = df['STD-YM'].unique()
# 빈 데이터프레임 생성
month_last_df = pd.DataFrame()
for ym in ym_list:
    data = df.loc[ym].tail(1)
    month_last_df = pd.concat([month_last_df, data], axis=0)

In [19]:
month_last_df

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1980-12-31 00:00:00+00:00,0.611607,0.611607,0.609375,0.609375,0.487276,8937600.0,1980-12
1981-01-30 00:00:00+00:00,0.508929,0.508929,0.504464,0.504464,0.403385,11547200.0,1981-01
1981-02-27 00:00:00+00:00,0.473214,0.477679,0.473214,0.473214,0.378397,3690400.0,1981-02
1981-03-31 00:00:00+00:00,0.441964,0.441964,0.437500,0.437500,0.349839,3998400.0,1981-03
1981-04-30 00:00:00+00:00,0.506696,0.511161,0.506696,0.506696,0.405170,3152800.0,1981-04
...,...,...,...,...,...,...,...
2019-02-28 00:00:00+00:00,174.320007,174.910004,172.919998,173.149994,172.485748,28215400.0,2019-02
2019-03-29 00:00:00+00:00,189.830002,190.080002,188.539993,189.949997,189.221313,23564000.0,2019-03
2019-04-30 00:00:00+00:00,203.059998,203.399994,199.110001,200.669998,199.900192,46534900.0,2019-04


In [20]:
# month_last_df에서 
# 전월(인덱스가 1칸 밑으로 이동된 데이터)의 수정종가 데이터를 BF_1M에 대입 
# 전년도(인덱스가 12칸 밑으로 이동된 데이터)의 수정종가 데이터를 BF_12M에 대입 
month_last_df['BF_1M'] = month_last_df.shift(1)['Adj Close']
month_last_df['BF_12M'] = month_last_df.shift(12)['Adj Close']

In [ ]:
month_last_df.iloc[10:15]

#### 거래내역 생성 
- 모멘텀 인덱스 : (전월의 수정종가 / 전년도의 수정종가) - 1
- 모멘텀 인덱스가 0보다는 크고 무한대가 아닌경우 -> 매수 

In [ ]:
# 거래내역은 df에 생성
df['trade'] = ''

for idx in month_last_df.index:
    signal = ''

    # 절대 모멘텀 계산식
    momentum_index = month_last_df.loc[idx, 'BF_1M'] / \
          month_last_df.loc[idx, 'BF_12M'] - 1
    flag = (momentum_index > 0) & (momentum_index != np.inf)
    # flag가 True인 경우 -> 매수
    if flag:
        signal = 'buy'
    # 2010-01에 signal이 buy 측정이 되면 -> 2010-01 말일에 보유 2월 말일 전까지 
    print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
    df.loc[idx:, 'trade'] = signal

In [23]:
df['trade'].value_counts()

trade
buy    6092
       3623
Name: count, dtype: int64

In [25]:
df.to_csv('test1.csv')
month_last_df.to_csv('test2.csv')

In [ ]:
# 수익율 계산
df['rtn'] = 1

for idx in df.index:
    # 매수 조건식 : 전날의 trade가 '' 오늘의 trade가 'buy'
    if (df.shift().loc[idx, 'trade'] == '') & \
        (df.loc[idx, 'trade'] == 'buy'):
        buy = df.loc[idx, 'Adj Close']
        print(f"매수일 : {idx}, 매수가 : {buy}")
    # 매도 조건식 : 전날의 trade가 'buy' 오늘의 trade가 ''
    elif (df.shift().loc[idx, 'trade'] == 'buy') & \
        (df.loc[idx, 'trade'] == ''):
        sell = df.loc[idx, 'Adj Close']
        print(f"매도일 : {idx}, 매도가 : {sell}")
        # 수익율 계산
        rtn = sell / buy
        df.loc[idx, 'rtn'] = rtn
        print(f"수익율 : {rtn}")

In [27]:
df['acc_rtn'] = df['rtn'].cumprod()
print(df.iloc[-1, -1])

75.26608134071219


### 절대 모멘텀 함수화 
1. create_ym 함수 
    - 매개변수 2개 
        - 데이터프레임 (_df)
        - 기준이 되는 컬럼의 이름 (_col = 'Adj Close')
    - 데이터프레임의 복사본을 생성해서 작업 
    - 컬럼에 Date가 존재하는가? 존재한다면 Date를 인덱스로 변환
    - index를 시계열 데이터로 변경
    - 데이터 중 결측치나 양의 무한대, 음의 무한대를 제외
    - 기준이되는 컬럼을 제외하고 모두 삭제
    - 'STD-YM' 파생변수를 생성하여 index에서 '년-월'을 추출하여 대입
    - 위에서 작업된 데이터프레임을 되돌려준다. 

In [29]:
def create_ym(
        _df, 
        _col = 'Adj Close'
):
    # 복사본 생성 
    result = _df.copy()
    # Date 컬럼이 존재하는가?
    if 'Date' in result.columns:
        # Date를 인덱스로 변경
        result.set_index('Date', inplace=True)
    # index를 시계열로 변경
    result.index = pd.to_datetime(result.index, utc=True)
    # 결측치 무한대 값들은 제외
    flag = result.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
    result = result.loc[~flag, [_col] ]
    # 파생변수를 생성 
    result['STD-YM'] = result.index.strftime('%Y-%m')
    return result
    

In [30]:
amzn_df = pd.read_csv("../../csv/AMZN.csv")

In [32]:
ym_df = create_ym(amzn_df)

- 두번째 함수 생성 (create_month)
    - 매개변수 5개 
        - STD-YM 컬럼이 존재하는 데이터프레임 (_df)
        - 시작 시간 (_start = '2010-01-01')
        - 종료 시간 (_end = datetime.now() )
        - 모멘텀 기간 ( _momentum = 12 )
        - 기준시점[월말, 월초] ( _select = 1 ) : 1인 경우 월말, 0인 경우 월초
    - _select가 1이라면 
        - _df에서 월말의 데이터들을 result에 대입
    - 0이라면
        - _df에서 월초의 데이터들을 result에 대입
    - 둘다 아니라면 
        - return을 이용해서 함수 종료, print를 이용하여 select 타입이 잘못되었다 출력
    - 기준이 되는 컬럼의 이름[_df에서 첫번째 컬럼의 이름]을 변수에 저장
    - 전월의 기준이되는 컬럼의 값을 BF1 컬럼에 대입 
    - _momentum 개월 전 데이터를 BF2 컬럼에 대입 
    - 시작시간과 종료시간을 기준으로 인덱스 필터링 
    - 결과(result)를 되돌려준다.  : ym_df는 변경이 되지 않은 상태


In [40]:
def create_month(
        _df, 
        _start = '2010-01-01', 
        _end = datetime.now(), 
        _momentum = 12, 
        _select = 1
):
    # _select가 1과 같다면
    if _select == 1:
        # 월말 데이터의 조건식을 생성
        flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
        # result = _df.loc[flag]
    elif _select == 0:
        # 월초 데이터의 조건식 생성
        flag = _df['STD-YM'] != _df.shift(1)['STD-YM']
        # result = _df.loc[flag]
    else:
        return "_select의 값은 0 아니면 1 입니다."
    result = _df.loc[flag]
    # 기준이 되는 컬럼의 이름을 변수에 저장 
    col = result.columns[0]
    # BF1컬럼을 생성
    # 전월의 데이터를 대입
    result['BF1'] = result.shift(1)[col]
    # _momentum 값의 과거의 개월수 데이터 대입 
    result['BF2'] = result.shift(_momentum)[col]
    try:
        result.index = result.index.tz_localize(None)
    except Exception as e:
        print(e)
    # 시작 시간과 종료시간을 기준으로 데이터 필터링 
    result = result.loc[_start : _end]
    return result
    
    

In [42]:
# warning 메시지 출력 금지 
import warnings
warnings.filterwarnings('ignore')

In [47]:
month_df = create_month(ym_df, _select=1)

- 세번째 함수 
    - 매개변수 3개 
        - trade 컬럼이 추가될 데이터프레임(_df1) -> create_ym의 결과값
        - 월말/월초의 데이터프레임(_df2) -> create_month함수의 결과값
        - 모멘텀 스코어 (_score = 1)
    - _df1의 복사본을 생성 (result)
    - result에 trade 컬럼을 생성하고 '' 대입
    - 반복문을 이용하여 구매내역 추가 
        - _df2 데이터프레임 이용하여 momentum_index를 생성
            - BF1 / BF2 - _socre
        - 구매 신호 (signal)을 생성
        - result에 있는 trade 컬럼에 buy를 대입
    - 구매내역이 추가된 데이터프레임을 되돌려준다. 

In [51]:
def create_trade(
        _df1, 
        _df2, 
        _score = 1
):
    # 복사본 생성
    result = _df1.copy()
    # trade 컬럼을 추가 
    result['trade'] = ''
    try:
        result.index = result.index.tz_localize(None)
    except Exception as e:
        print(e)

    # 반복문 생성 -> _df2를 기준으로 반복
    for idx in _df2.index:
        signal = ''
        # 모멘텀 인덱스를 계산
        momentum_index = _df2.loc[idx, 'BF1'] / \
              _df2.loc[idx,'BF2'] - _score
        # 모멘텀 인덱스를 이용하여 구매 신호 생성
        flag = (momentum_index > 0) & (momentum_index != np.inf)
        if flag :
            signal = 'buy'
        # 출력 
        print(f"날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}")
        # 구매 신호를 result에 대입 
        result.loc[idx:, 'trade'] = signal
    
    return result


In [ ]:
trade_df = create_trade(ym_df, month_df)

In [54]:
# 수익율을 계산하는 함수 
def create_rtn(_df):
    # 복사본 생성 
    result = _df.copy()
    # 기준이 되는 컬럼의 이름 
    col = result.columns[0]

    # rtn 컬럼을 생성 1을 대입 
    result['rtn'] = 1

    for idx in result.index:
        # 매수 
        if (result.shift().loc[idx, 'trade'] == '') &\
            (result.loc[idx, 'trade'] == 'buy'):
            buy = result.loc[idx, col]
            print(f"매수일 : {idx}, 매수가 : {buy}")
        # 매도
        elif (result.shift().loc[idx, 'trade'] == 'buy') &\
            (result.loc[idx, 'trade'] == ''):
            sell = result.loc[idx, col]
            print(f"매도일 : {idx}, 매도가 : {sell}")
            # 수익율 계산
            rtn = sell / buy
            # 수익율을 데이터프레임에 대입 
            result.loc[idx, 'rtn'] = rtn
            print(f"수익율 : {rtn}")
    # 누적수익율 계산하고 대입 
    result['acc_rtn'] = result['rtn'].cumprod()
    # 총 누적수익율을 변수에 저장 
    acc_rtn = result.iloc[-1, -1]
    return result, acc_rtn


In [55]:
create_rtn(trade_df)

매수일 : 2010-01-29 00:00:00, 매수가 : 125.410004
매도일 : 2012-03-30 00:00:00, 매도가 : 202.509995
수익율 : 1.6147834187135501
매수일 : 2012-04-30 00:00:00, 매수가 : 231.899994
매도일 : 2014-10-31 00:00:00, 매도가 : 305.459991
수익율 : 1.31720568737919
매수일 : 2015-03-31 00:00:00, 매수가 : 372.100006


(              Adj Close   STD-YM trade  rtn   acc_rtn
 Date                                                 
 1997-05-15     1.958333  1997-05        1.0  1.000000
 1997-05-16     1.729167  1997-05        1.0  1.000000
 1997-05-19     1.708333  1997-05        1.0  1.000000
 1997-05-20     1.635417  1997-05        1.0  1.000000
 1997-05-21     1.427083  1997-05        1.0  1.000000
 ...                 ...      ...   ...  ...       ...
 2019-06-18  1901.369995  2019-06   buy  1.0  2.127002
 2019-06-19  1908.790039  2019-06   buy  1.0  2.127002
 2019-06-20  1918.189941  2019-06   buy  1.0  2.127002
 2019-06-21  1911.300049  2019-06   buy  1.0  2.127002
 2019-06-24  1907.953857  2019-06   buy  1.0  2.127002
 
 [5563 rows x 5 columns],
 2.1270019030151)